In [14]:
import logging
import ray
from ray import tune
import ray.tune.progress_reporter as reporter
from typing import Optional, Type
from ray.rllib.models.torch.torch_modelv2 import TorchModelV2
from ray.rllib.models.torch.misc import normc_initializer, SlimFC, SlimConv2d
from ray.rllib.utils.annotations import override
from ray.rllib.utils.framework import try_import_torch
from ray.rllib.utils.typing import ModelConfigDict
import ray.rllib.agents.ddpg as ddpg

In [2]:
torch, nn = try_import_torch()
ray.init()


2021-08-12 12:57:12,306	INFO services.py:1245 -- View the Ray dashboard at http://127.0.0.1:8265


{'node_ip_address': '10.105.173.195',
 'raylet_ip_address': '10.105.173.195',
 'redis_address': '10.105.173.195:6379',
 'object_store_address': '/tmp/ray/session_2021-08-12_12-57-10_308146_19459/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2021-08-12_12-57-10_308146_19459/sockets/raylet',
 'webui_url': '127.0.0.1:8265',
 'session_dir': '/tmp/ray/session_2021-08-12_12-57-10_308146_19459',
 'metrics_export_port': 52760,
 'node_id': 'd011cf18ec26d65fa574e792aaa9348b5dcb116646829b3a7a9159c5'}

In [3]:
#######################################################
###### currently use the default model, which is the fully connected and linear layers
#######################################################

model = {"fcnet_hiddens": [200, 100], "fcnet_activation": "tanh"}

In [8]:
#######################################################
##### DDPG config
#######################################################

config = ddpg.DEFAULT_CONFIG.copy()
train_batch_size = 200  ## batch size will be tuned later

## customized config

config_ddpg = {
    
    "num_gpus": 4,
    "lr": 1e-5, ## starting low
    "monitor": True,
    "model":model,
    "train_batch_size":train_batch_size,
    "lambda":0.95,
    "clip_actions":True,
    "normalize_actions":True,
    "batch_mode": "complete_episodes", ## or "truncate_episodes"
    "no_done_at_end":True,
    "shuffle_buffer_size": 200, ## need to be tuned
    "framework": "torch",
    "corrifor_length": 1,
#     "env": environment, ## need to be import from custom
    
    
}
config.update(config_ddpg)


In [ ]:
class Corridor(gym.Env):
    """Example of a custom env in which you have to walk down a corridor.
    You can configure the length of the corridor via the env config."""

    def __init__(self, config: EnvContext):
        self.end_pos = config["corridor_length"]
        self.cur_pos = 0
        self.action_space = Discrete(2)
        self.observation_space = Box(
            0.0, self.end_pos, shape=(1, ), dtype=np.float32)
        # Set the seed. This is only used for the final (reach goal) reward.
        self.seed(config.worker_index * config.num_workers)

    def reset(self):
        self.cur_pos = 0
        return [self.cur_pos]

    def step(self, action):
        assert action in [0, 1], action
        if action == 0 and self.cur_pos > 0:
            self.cur_pos -= 1
        elif action == 1:
            self.cur_pos += 1
        done = self.cur_pos >= self.end_pos
        # Produce a random reward when we reach the goal.
        return [self.cur_pos], \
            random.random() * 2 if done else -0.1, done, {}

    def seed(self, seed=None):
        random.seed(seed)

In [20]:
### define model
## only the template
class Net(TorchModelV2, nn.Module):
    def __init__(self, obs_space: ,
                 action_space: ,
                 num_outputs: int,
                 ModelConfigDict, name:str):
        
        TorchModelV2.__init__(self, obs_space, action_space, num_outputs,
                              model_config, name)
        nn.Module.__init__(self)
        custom_configs = model_config.get("custom_model_config")
        self.flatten = nn.Flatten()
        grasp_num = 3  ## need to be specified
        self.ls = nn.LSTM(32,64,2, batch_first = True)
        self.linear = nn.Linear(64,grasp_num + 7)
            

    def forward(self, x):
        batch_size = x.shape[0] 
        x = self.flatten(x)
        x = torch.reshape(x, (batch_size, 100, 32)) 
        output,(h,c) = self.ls(x)
        output = self.linear(h[-1,...]) 
        return output ##nn.Sigmoid()(output)

model = Net()
print(model)

TypeError: __init__() missing 5 required positional arguments: 'obs_space', 'action_space', 'num_outputs', 'model_config', and 'name'

In [9]:
def stopper(trial_num, result):
    ### determine when the training will stop
    #(when mean reward larger than 50 or episode larger than 1500 will stop train)
    return result["episode_reward_mean"] > 50 and result["episode_len_mean"] > 1500

In [17]:
tune.run("DDPG", 
        config = config,
#         local_dir = SAVE_RESULTS_PATH, ## need to be set path when we trained on beast
        checkpoint_freq=25,
        verbose=2,
        checkpoint_at_end=True,
        stop=stopper,
#         progress_reporter=reporter ## need to be set later
        )

2021-08-12 13:29:09,957	WARNING tune.py:507 -- SIGINT received (e.g. via Ctrl+C), ending Ray Tune run. This will try to checkpoint the experiment state one last time. Press CTRL+C one more time (or send SIGINT/SIGKILL/SIGTERM) to skip. 


Trial name,status,loc
DDPG_None_258bd_00000,PENDING,


2021-08-12 13:29:10,168	ERROR tune.py:546 -- Trials did not complete: [DDPG_None_258bd_00000]
2021-08-12 13:29:10,169	INFO tune.py:550 -- Total run time: 16.84 seconds (16.62 seconds for the tuning loop).
2021-08-12 13:29:10,170	WARNING tune.py:554 -- Experiment has been interrupted, but the most recent state was saved. You can continue running this experiment by passing `resume=True` to `tune.run()`
